In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.FactorizationMachines import LightFMRecommender

model = LightFMRecommender.LightFMCFRecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 2604 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [3]:
import optuna as op

class Objective:
    def __init__(self):
        self.step = 1.0

    def __call__(self, trial: op.Trial):
        self.step = 1.0
        trial.suggest_int("n_components", 1, 200)
        trial.suggest_categorical("loss", ['bpr', 'warp', 'warp-kos'])
        trial.suggest_categorical("sgd_mode", ['adagrad', 'adadelta'])
        trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True)
        trial.suggest_float("item_alpha", 1e-5, 1e-2, log=True)
        trial.suggest_float("user_alpha", 1e-5, 1e-2, log=True)

        def callback_prune(value):
            if value is None:
                return
            trial.report(value, step=self.step)
            self.step += 1

        recommender = model(URM_train)
        recommender.fit(**trial.params, epochs=100, num_threads=7, stop_on_validation=True, validation_metric="MAP",
                        validation_every_n=10,
                        lower_validations_allowed=2,
                        evaluator_object=evaluator_validation, callback_validation=callback_prune)

        result, _ = evaluator_validation.evaluateRecommender(recommender)
        MAP_result = result["MAP"].item()

        return MAP_result


In [ ]:
study_name = "light-fm"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
# study.enqueue_trial(best_params)
objective = Objective()
study.optimize(objective, n_trials=10)

[I 2023-12-09 17:32:46,949] Using an existing study with name 'light-fm' instead of creating a new one.


LightFMCFRecommender: URM Detected 458 ( 3.6%) users with no interactions.
LightFMCFRecommender: URM Detected 316 ( 1.4%) items with no interactions.
LightFMCFRecommender: Epoch 1 of 100. Elapsed time 0.23 sec
LightFMCFRecommender: Epoch 2 of 100. Elapsed time 0.45 sec
LightFMCFRecommender: Epoch 3 of 100. Elapsed time 0.67 sec
LightFMCFRecommender: Epoch 4 of 100. Elapsed time 0.87 sec
LightFMCFRecommender: Epoch 5 of 100. Elapsed time 1.08 sec
LightFMCFRecommender: Epoch 6 of 100. Elapsed time 1.29 sec
LightFMCFRecommender: Epoch 7 of 100. Elapsed time 1.50 sec
LightFMCFRecommender: Epoch 8 of 100. Elapsed time 1.70 sec
LightFMCFRecommender: Epoch 9 of 100. Elapsed time 1.91 sec
LightFMCFRecommender: Validation begins...


In [9]:
study.best_params

{'topK': 8894, 'l1_ratio': 0.05565733019999427, 'alpha': 0.0012979360257937668}

In [10]:
final = model(URM_train_validation)
final.fit(**study.best_params)

SLIMElasticNetRecommender: URM Detected 221 ( 1.7%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 113 ( 0.5%) items with no interactions.


100%|█████████▉| 22200/22222 [03:20<00:00, 68.60it/s] 

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2191 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10447 (100.0%) in 6.49 sec. Users per second: 1611


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.094821                 0.163454  0.137526  0.048307     0.08229   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.267329  0.148903  0.112249  0.516129      0.355594  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.826634           0.42665    0.826634       0.055048   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.183917                   0.997667             0.157923   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   